# Анализ данных сервиса для чтения книг

# Оглавление <a name="оглавление">

1. [Знакомство с данными](#данные)  
2. [Задачи](#анализ)  
    2.1. [Задача №1](#задача_1)  
    2.2. [Задача №2](#задача_2)  
    2.3. [Задача №3](#задача_3)  
    2.4. [Задача №4](#задача_4)  
    2.5. [Задача №5](#задача_5)


### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [1]:
import pandas as pd
from sqlalchemy import create_engine


In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных


In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])



In [4]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [5]:
def slct(query):
    display(pd.io.sql.read_sql(query, con = engine))

# 1. Посмотрим, как выглядят исходные данные <a name="данные"></a>

In [6]:
books = ''' SELECT * FROM books'''

In [7]:
slct(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


### В таблице books 1000 записей. Посмотрим, есть ли повторяющиеся книги

In [8]:
duplicates = ''' 
SELECT * 
FROM books
WHERE title IN 
    (SELECT title 
    FROM books
    GROUP BY title
    HAVING COUNT(book_id) > 1) '''

In [9]:
slct(duplicates)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,426,39,Memoirs of a Geisha,434,2005-11-15,241
1,427,39,Memoirs of a Geisha,503,2005-11-22,311


#### Есть две книги с одинаковым названием, но разных издательств и с разным кол-вом страниц, так что это нельзя назвать дубликатами

In [10]:
authors = ''' SELECT * FROM authors'''

In [11]:
slct(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
...,...,...
631,632,William Strunk Jr./E.B. White
632,633,Zadie Smith
633,634,Zilpha Keatley Snyder
634,635,Zora Neale Hurston


In [12]:
publishers = ''' SELECT * FROM publishers'''

In [13]:
slct(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
...,...,...
335,336,Workman Publishing Company
336,337,Wyatt Book
337,338,Yale University Press
338,339,Yearling


In [14]:
ratings = ''' SELECT * FROM ratings'''

In [15]:
slct(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
...,...,...,...,...
6451,6452,1000,carolrodriguez,4
6452,6453,1000,wendy18,4
6453,6454,1000,jarvispaul,5
6454,6455,1000,zross,2


In [16]:
reviews = ''' SELECT * FROM reviews'''

In [17]:
slct(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
...,...,...,...,...
2788,2789,999,martinadam,Later hospital turn easy community. Fact same ...
2789,2790,1000,wknight,Change lose answer close pressure. Spend so now.
2790,2791,1000,carolrodriguez,Authority go who television entire hair guy po...
2791,2792,1000,wendy18,Or western offer wonder ask. More hear phone f...


#### [вернуться к оглавлению](#оглавление)

# 2. Решение задач <a name="задачи"></a>

## Посчитайте, сколько книг вышло после 1 января 2000 года <a name="задача_1"></a>

In [18]:
query1 = ''' SELECT COUNT(book_id) 
FROM books 
WHERE CAST(publication_date AS date) > '2000-01-01' ''' 

In [19]:
slct(query1) 

,count
0,819


#### После 1 января 2000 года вышло 819 книг

#### [вернуться к оглавлению](#оглавление)

## Для каждой книги посчитайте количество обзоров и среднюю оценку <a name="задача_2"></a>

In [20]:
query2 = ''' SELECT book_id, title, average_rating, 
CASE WHEN review_count IS NULL THEN 0 ELSE review_count END
FROM books 
LEFT JOIN (SELECT book_id, COUNT(review_id) AS review_count
FROM reviews
GROUP BY book_id) as count_reviews USING(book_id) 
LEFT JOIN (SELECT book_id, ROUND(AVG(rating), 2) AS average_rating
FROM ratings
GROUP BY book_id) as avg_rating USING(book_id)

'''


In [21]:
slct(query2)

,book_id,title,average_rating,review_count
0,652,The Body in the Library (Miss Marple #3),4.50,2
1,273,Galápagos,4.50,2
2,51,A Tree Grows in Brooklyn,4.25,5
3,951,Undaunted Courage: The Pioneering First Missio...,4.00,2
4,839,The Prophet,4.29,4
...,...,...,...,...
995,64,Alice in Wonderland,4.23,4
996,55,A Woman of Substance (Emma Harte Saga #1),5.00,2
997,148,Christine,3.43,3
998,790,The Magicians' Guild (Black Magician Trilogy #1),3.50,2


#### В полученной таблице содержатся данные о кол-ве обзоров и средней оценке для каждой книги

#### [вернуться к оглавлению](#оглавление)

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры <a name="задача_3"></a>

In [22]:
query3 = ''' SELECT publishers.publisher, COUNT(books.book_id) AS cnt
FROM books 
LEFT JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher 
ORDER BY cnt DESC
LIMIT 1
'''

In [23]:
slct(query3)

,publisher,cnt
0,Penguin Books,42


#### Издательство Penguin Books выпустило 42 книги толще 50 страниц 

#### [вернуться к оглавлению](#оглавление)

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками <a name="задача_4"></a>

In [24]:
query4 = '''SELECT authors.author, ROUND(AVG(ratings.rating), 2)
FROM books
LEFT JOIN authors ON books.author_id = authors.author_id 
LEFT JOIN ratings ON books.book_id = ratings.book_id
WHERE books.book_id IN 
          (SELECT book_id
            FROM ratings 
              GROUP BY book_id
              HAVING COUNT(rating_id) > 50)
GROUP BY authors.author
ORDER BY AVG(ratings.rating) DESC
LIMIT 1
'''

In [25]:
slct(query4)

,author,round
0,J.K. Rowling/Mary GrandPré,4.29


#### ***J.K. Rowling*** - автор с самой высокой средней оценкой (4,29)

#### [вернуться к оглавлению](#оглавление)

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок <a name="задача_5"></a>

In [26]:
query5 = ''' SELECT ROUND(AVG(cnt)) AS avg_cnt
FROM (SELECT username, COUNT(review_id) as cnt
      FROM reviews
      WHERE username IN (SELECT username
                                 FROM ratings 
                                  GROUP BY username
                                  HAVING COUNT(rating_id) > 50)
      GROUP BY username) as subquery                        
'''


In [27]:
slct(query5)

,avg_cnt
0,24.0


#### Пользователи, поставившие более 50 оценок в среднем пишут 24 обзора

#### [вернуться к оглавлению](#оглавление)